In [2]:
import ollama

dymmy_data = { # this is the profile of the user
    'name': 'John Doe',
    'email': 'example@mail.com',
    'telephone': '1234567890',
    'address': '1234 Main Street, City, Country',
    'Overview': 'I am John Doe, an autistic individual with a strong passion for computers. I have developed several skills in programming, troubleshooting, and software development. My keen attention to detail and problem-solving abilities make me proficient in coding languages and various software applications. I am eager to leverage my technical skills to contribute effectively in a tech-driven environment.0',
    'skills': ['Python', 'Java', 'C++', 'JavaScript', 'HTML', 'CSS'],
    'disability': 'Autism',
    'social': "https://www.linkedin.com/in/johndoe",
    'work': [
        {
            'title': 'Software Developer',
            'company': 'ABC Inc.',
            'start_date': 'January 2020',
            'end_date': 'Present',
            'description': 'Developed software applications using Python, Java, and C++.'
        },
        {
            'title': 'Technical Support Specialist',
            'company': 'XYZ Corp.',
            'start_date': 'June 2018',
            'end_date': 'December 2019',
            'description': 'Provided technical support to customers and resolved software issues.'
        }
    ],
    'education': [
        {
            'degree': 'Bachelor of Science in Computer Science',
            'school': 'University of ABC',
            'graduation_date': 'May 2018',
            'start_date': 'September 2014',
        }
    ],
}

def parce_profile(profile):
    # parse the profile and return the text
    text = profile['Overview']
    for work in profile['work']:
        text += f" {work['title']} at {work['company']} from {work['start_date']} to {work['end_date']}. {work['description']}"
    for education in profile['education']:
        text += f" {education['degree']} at {education['school']} from {education['start_date']} to {education['graduation_date']}"
    return text

# load the model mxbai-embed-large
embedding = ollama.embeddings(model='mxbai-embed-large', prompt=parce_profile(dymmy_data))["embedding"]
print(embedding)
# print(len(embedding)) # the length of the embedding is 1024

[0.28269243240356445, -0.2364664375782013, -0.28954294323921204, -0.3176056444644928, 0.10015943646430969, -0.25621265172958374, -0.28229913115501404, -0.25472211837768555, 0.4540674686431885, 0.5720822811126709, -0.3175504207611084, -0.6161708831787109, 0.6181646585464478, -0.15966370701789856, -0.08195162564516068, -0.4598086178302765, -0.18629363179206848, -0.3838425874710083, -1.061079740524292, 0.1062961146235466, 0.23890501260757446, 0.24099934101104736, -0.7964125871658325, -0.12457793951034546, -0.37022528052330017, 0.5442231297492981, 0.11936907470226288, -0.18674322962760925, 1.0336956977844238, 0.627404510974884, 0.006346814334392548, -0.2973657250404358, -0.203815758228302, -0.9626678228378296, 0.7081184983253479, -0.45346713066101074, 0.42090117931365967, -0.7623593211174011, -0.08562588691711426, -0.6535407900810242, 0.5872154831886292, -0.3504795730113983, -0.011031031608581543, -1.1568119525909424, -1.5858185291290283, -0.33968856930732727, 0.36103716492652893, 0.168483